In [1]:
# OBJETIVO:
# A partir dos datasets "acordaos2020" (NOGALI, 2021), "sumulas" (notebook _1_scraping_sumulas)
# e "precedentes" (notebook _3_xml_precedentes), realizar as transformações e integrações
# necessárias para produzir o dataset "corpus", insumo para os modelos de busca de similaridade.

In [2]:
import nltk
import numpy as np
import pandas as pd
import re
from nltk.text import Text
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
# baixa dados do NLTK
nltk.download('punkt')

In [3]:
# carrega os datasets
acordaos = pd.read_csv('datasets/acordaos2020.csv')
sumulas = pd.read_csv('datasets/sumulas.csv')
precedentes = pd.read_csv('datasets/precedentes.csv')

In [4]:
# estrutura do dataset "acordaos2020"
print('Registros de "acordaos2020":', len(acordaos))
print('Duplicidades de número de acórdao:', acordaos.duplicated(['Decisao']).sum())
print('\nPrimeiro registro (estrutura original):')
print(acordaos.iloc[0])

Registros de "acordaos2020": 35887
Duplicidades de número de acórdao: 12

Primeiro registro (estrutura original):
Unnamed: 0                                                        0
Processo                                       10026.720020/2016-31
Decisao                                                2201-006.064
TipoRecurso                                              Voluntário
Turma             2ª Seção de Julgamento / 2ª Câmara / 1ª Turma ...
Data                                                     04/02/2020
Recorrente        DENJUD REFEIÇÕES COLETIVAS ADMINISTRAÇÃO E SER...
Recorrida                                          FAZENDA NACIONAL
NomeRelator                                                        
Ementa            ASSUNTO: CONTRIBUIÇÕES SOCIAIS PREVIDENCIÁRIAS...
Acordao           Vistos, relatados e discutidos os presentes au...
Relatorio         Cuida-se de Recurso Voluntário de fls. 219/264...
Voto              Conselheiro Rodrigo Monteiro Loureiro Amorim, ...
Vo

In [5]:
# remove colunas de "acordaos2020" que não contribuem para a caracterização da tese discutida
acordaos.drop(columns=['Unnamed: 0', 'Processo', 'TipoRecurso', 'Turma', 'Data', 'Recorrente',
                       'Recorrida', 'NomeRelator', 'Acordao', 'DeclaracaoVoto'], inplace=True)
# renomeia a coluna 'Decisao' (contém o número do acórdão) para 'Acordao'
acordaos.rename(columns={'Decisao':'Acordao'}, inplace=True)
print('\nPrimeiro registro (estrutura ajustada):')
print(acordaos.iloc[0])


Primeiro registro (estrutura ajustada):
Acordao                                              2201-006.064
Ementa          ASSUNTO: CONTRIBUIÇÕES SOCIAIS PREVIDENCIÁRIAS...
Relatorio       Cuida-se de Recurso Voluntário de fls. 219/264...
Voto            Conselheiro Rodrigo Monteiro Loureiro Amorim, ...
VotoVencedor                                                   \n
Name: 0, dtype: object


In [6]:
# identifica duplicidades de número de acórdão
acordaos.loc[acordaos.duplicated(['Acordao'], keep=False)].sort_values(['Acordao'])

,Acordao,Ementa,Relatorio,Voto,VotoVencedor
22452,1002-001.604,ASSUNTO: PROCESSO ADMINISTRATIVO FISCAL\nAno-c...,Trata­se de Recurso Voluntário interposto ...,"Conselheiro Rafael Zedral, Relator.\nAdmissibi...",\n
22451,1002-001.604,ASSUNTO: PROCESSO ADMINISTRATIVO FISCAL\nAno-c...,Trata­se de Recurso Voluntário interposto ...,"Conselheiro Rafael Zedral, Relator.\nAdmissibi...",\n
30501,1402-005.182,ASSUNTO: OBRIGAÇÕES ACESSÓRIAS\nAno-calendário...,Trata o presente de Recurso Voluntário interpo...,"Conselheiro Marco Rogério Borges, Relator.\n\n...",\n
17049,1402-005.182,ASSUNTO: OBRIGAÇÕES ACESSÓRIAS\nAno-calendário...,Trata o presente de Recurso Voluntário interpo...,"\nConselheiro Marco Rogério Borges, Relator.\n...",\n\n
5098,2002-002.067,ASSUNTO: OBRIGAÇÕES ACESSÓRIAS\nAno-calendário...,O presente julgamento submete-se à sistemática...,Conselheira Claudia Cristina Noira Passos da C...,\n
21986,2002-002.067,ASSUNTO: PROCESSO ADMINISTRATIVO FISCAL\nAno-c...,Trata-se de notificação de lançamento lavrada ...,"Conselheiro André Luis Ulrich Pinto, Relator.\...",\n
5714,2002-002.099,ASSUNTO: OBRIGAÇÕES ACESSÓRIAS\nAno-calendário...,O presente julgamento submete-se à sistemática...,Conselheira Claudia Cristina Noira Passos da C...,\n
8486,2002-002.099,ASSUNTO: IMPOSTO SOBRE A RENDA DE PESSOA FÍSIC...,Trata-se de notificação de lançamento lavrada ...,"Conselheiro André Luis Ulrich Pinto, Relator.\...",\n
13701,2201-006.090,NaN,O presente julgamento submete-se à sistemática...,"Conselheiro Carlos Alberto do Amaral Azeredo, ...",\n
13704,2201-006.090,ASSUNTO: IMPOSTO SOBRE A PROPRIEDADE TERRITORI...,Trata-se de Recurso Voluntário da decisão prof...,"Conselheiro Douglas Kakazu Kushiyama, Relator....",\n


In [7]:
# remove duplicidades
acordaos.drop_duplicates(['Acordao'], keep=False, inplace=True)
# substitui nulos por vazio
acordaos.fillna('', inplace=True)
# substitui "brancos" por vazio
acordaos.replace(r'^\s+$', '', regex=True, inplace=True)
# contabiliza os vazios
print('Registros:', len(acordaos))
print('Vazios:')
print((acordaos == '').sum())

Registros: 35863
Vazios:
Acordao             0
Ementa            949
Relatorio           0
Voto                0
VotoVencedor    34688
dtype: int64


In [8]:
# estrutura do dataset "sumulas"
print('Registros de "sumulas":', len(sumulas))
print('Duplicidades de número de súmula:', sumulas.duplicated(['Sumula']).sum())
print('\nPrimeiro registro:')
print(sumulas.iloc[0])

Registros de "sumulas": 158
Duplicidades de número de súmula: 0

Primeiro registro:
Sumula                                                         1
Enunciado      Importa renúncia às instâncias administrativas...
Precedentes    101-93877 103-21884 105-14637 107-06963 108-07...
Name: 0, dtype: object


In [9]:
# estrutura do dataset "precedentes"
print('Registros de "precedentes":', len(precedentes))
print('Duplicidades de número de acórdao:', precedentes.duplicated(['Acordao']).sum())
print('\nPrimeiro registro:')
print(precedentes.iloc[0])

Registros de "precedentes": 405
Duplicidades de número de acórdao: 0

Primeiro registro:
Acordao                                              1101-000.627
Ementa          Assunto: Imposto sobre a Renda de Pessoa Juríd...
Relatorio       A empresa acima identificada foi autuada e not...
Voto            Conselheiro BENEDICTO CELSO BENÍCIO JÚNIOR, Re...
VotoVencedor    Conselheira EDELI PEREIRA BESSA\nO presente vo...
Name: 0, dtype: object


In [10]:
# substitui nulos por vazio
precedentes.fillna('', inplace=True)
# substitui "brancos" por vazio
precedentes.replace(r'^\s+$', '', regex=True, inplace=True)
# contabiliza os vazios
print('Vazios:')
print((precedentes == '').sum())

Vazios:
Acordao           0
Ementa            0
Relatorio         0
Voto              0
VotoVencedor    326
dtype: int64


In [11]:
# função para obter uma lista de números de súmula de que um acórdão é precedente
def get_sumulas(num_acordao):
    # remove o '.' antes de comparar
    num_acordao = re.sub(r'\.', '', num_acordao)
    df = sumulas.loc[sumulas['Precedentes'].str.replace(r'\.', '', regex=True).str.contains(num_acordao)]
    return df['Sumula'].tolist()

# adiciona a  coluna "SumulaDerivada", contendo o número da súmula de que o acórdão foi precedente
sumulas_derivadas = precedentes['Acordao'].apply(get_sumulas)
precedentes['SumulaDerivada'] = sumulas_derivadas
# verifica se há precedentes de mais de uma súmula
precs_mais_uma = precedentes[[len(lista) > 1 for lista in sumulas_derivadas]]
# descarta os precedentes de mais de uma súmula
precedentes.drop(precs_mais_uma.index, inplace=True)
precs_mais_uma

,Acordao,Ementa,Relatorio,Voto,VotoVencedor,SumulaDerivada
163,3102-002.141,Assunto: Contribuição de Intervenção no Domíni...,Em face do encerramento do mandato do Conselhe...,"Conselheiro José Fernandes do Nascimento, Reda...",A divergência que deu azo à designação desse C...,"[127, 158]"
235,9101-002.245,Assunto: Imposto sobre a Renda de Pessoa Juríd...,Trata-se de processo originado pela lavratura ...,Conselheira Cristiane Silva Costa\n\nO conheci...,,"[135, 138]"
270,9101-003.376,Assunto: Imposto sobre a Renda de Pessoa Juríd...,GUIDE INVESTIMENTOS S.A. CORRETORA DE VALORES ...,"Conselheira Adriana Gomes Rêgo, Relatora.\nO r...",,"[108, 118]"
364,9303-005.293,Assunto: Contribuição de Intervenção no Domíni...,Trata-se de recurso especial de divergência in...,Conselheira Vanessa Marini Cecconello - Relato...,Conselheiro Demes Brito - Redator designado\nE...,"[108, 158]"


In [12]:
# cria o dataset "corpus", concatenando os datasets "acordaos2020" e "precedentes"
corpus = pd.concat([acordaos, precedentes], ignore_index=True)
# transforma a coluna 'SumulaDerivada' para inteiro, onde 0=não tem súmula derivada
corpus['SumulaDerivada'] = [0 if lista is np.nan else lista[0] for lista in corpus['SumulaDerivada']]
# concatena em "Documento" os campos "Ementa", "Relatorio" e "Voto" (se não houver "VotoVencedor")
# ou "VotoVencedor" (se houver), utilizando '\n' como separador
corpus['Documento'] = corpus['Ementa'].str.cat(corpus['Relatorio'], sep='\n').str.cat(
    np.where(corpus['VotoVencedor'], corpus['VotoVencedor'], corpus['Voto']), sep='\n')
# remove os campos "Ementa", "Relatorio", "Voto" e "VotoVencedor"
corpus.drop(columns=['Ementa', 'Relatorio', 'Voto', 'VotoVencedor'], inplace=True)
print('Registros de "corpus":', len(corpus))
print('Acórdãos precedentes de súmula no "corpus":', np.count_nonzero(corpus['SumulaDerivada']))
corpus[-403:-399]

Registros de "corpus": 36264
Acórdãos precedentes de súmula no "corpus": 401


,Acordao,SumulaDerivada,Documento
35861,2301-006.621,0,ASSUNTO: IMPOSTO SOBRE A RENDA DE PESSOA FÍSIC...
35862,2301-006.714,0,ASSUNTO: CONTRIBUIÇÕES SOCIAIS PREVIDENCIÁRIAS...
35863,1101-000.627,103,Assunto: Imposto sobre a Renda de Pessoa Juríd...
35864,1101-000.961,116,Assunto: Processo Administrativo Fiscal\nAno-c...


In [ ]:
%%time
# EXEMPLO: explora ocorrências de determinada palavra no texto, exibindo o contexto vizinho
docs = corpus['Documento'].sample(1000) # amostrado para limitar o tempo de processamento...
tokens = [token for doc in docs for token in word_tokenize(doc, language='portuguese')]
print('Tokens na amostra:', len(tokens))
text = Text(tokens)
text.concordance('lei')

In [13]:
# função para substituir determinadas expressões de texto
# (veja o propósito de cada expressão regular nos comentários da função)
s1 = re.compile(r'(?i)(lei|decreto|decreto\-lei) +(n\.?.? *)?(\d+)\.?(\d+)?(/\d+)?')
s2 = re.compile(r'(?i)lei +complementar +(n\.?.? *)?(\d+)\.?(\d+)?(/\d+)?')
s3 = re.compile(r'(?i)(portaria|port\.) +(conjunta +|interministerial +)?([a-z/]{2,}) +'
                r'(n\.?.? *)?(\d+)\.?(\d+)?(/\d+)?')
s4 = re.compile(r'(?i)(in|instrução normativa) +((srf|rfb) +)?(n\.?.? *)?(\d+)\.?(\d+)?(/\d+)?')
s5 = re.compile(r'(?i)s.mula +((do +|deste +)?carf +)(n\.?.? *)?(\d+)')
s6 = re.compile(r'(?i)s.mula +(n\.?.? *)?(\d+)( +(do +|deste +)?carf)')
s7 = re.compile(r'(?i)s.mula +vinculante +(n\.?.? *)?(\d+)\.?(\d+)?')
s8 = re.compile(r'(?i)(mp|medida +provisória) +(n\.?.? *)?(\d+)\.?([\-\d]+)(/\d+)?')
s9 = re.compile(r'(?i)arts?( ?\. *|igo +)(\d+)\.?(°|º|ª|o)?')
s10 = re.compile(r'[^_\d]\d[\d\-/\.,;\)]+') # não combina com 'lei_8212'
s11 = re.compile(r'\b\d\w*\b') # '_' não é word boundary: não combina com 'lei_8212'
s12 = re.compile(r'\b(e\-)?fls?\.|\bp\.')

def preprocess(text):
    # substitui, p. ex., "Lei n.º 8.212" por "lei_8212"
    text = re.sub(s1, r'\1_\3\4', text)
    # substitui, p. ex., "Lei Complementar nº 123" por "lei_complementar_123"
    text = re.sub(s2, r'lei_complementar_\2\3', text)
    # substitui, p. ex., "Portaria MF nº 343" por "port_MF_343"
    text = re.sub(s3, r'port_\3_\5\6', text)
    # substitui, p. ex., "IN RFB nº880" por "in_880"
    text = re.sub(s4, r'in_\5\6', text)
    # substitui, p. ex., "Súmula CARF n. 46" por "sumula_carf_46"
    text = re.sub(s5, r'sumula_carf_\4', text)
    # substitui, p. ex., "Súmula 46 deste CARF" por "sumula_carf_46"
    text = re.sub(s6, r'sumula_carf_\2', text)
    # substitui, p. ex., "Súmula Vinculante nº 8" por "sumula_vinculante_8"
    text = re.sub(s7, r'sumula_vinculante_\2\3', text)
    # substitui, p. ex., "Medida Provisória no 1.599-49" por "mp_1599-49"
    text = re.sub(s8, r'mp_\3\4', text)
    # substitui, p. ex., "art . 47-B" por "art_47-B"
    text = re.sub(s9, r'art_\2', text)
    # substitui por espaço sequências contendo apenas dígitos e determinados outros símbolos
    text = re.sub(s10, ' ', text)
    # substitui por vazio sequências iniciadas por dígito, p. ex., "1ª" ou "3ºcâmara"
    text = re.sub(s11, '', text)
    # substitui "f.", "fl.", "fls." e "p." por vazio
    text = re.sub(s12, '', text)
    return text

In [14]:
%%time
# aplica a função de pré-processamento aos documentos do corpus
corpus['Documento'] = corpus['Documento'].apply(preprocess)

Wall time: 3min 22s


In [15]:
# exporta o dataset "corpus"
corpus.to_csv('datasets/corpus.csv', index=False)